# Energy calculations with F-SAC

In this notebook, we use our SAC equations as the basis for F-SAC calculations.

The purpose of this notebook is to check the energy calculations by Eq. (35) for
the case of temperature dependent interaction energies.

In [42]:
import os, sys, math
import matplotlib.pyplot as plt
import pandas as pd

parent_directory = os.path.dirname(os.path.abspath('.'))
sys.path.append(parent_directory)

from pysac import SAC

### Compound definitions

In F-SAC, compounds are defined with just a few surface areas and their respective charge densities.
For a mixture of *n*-hexane and acetone it would be:

In [43]:
sigma_1 = [0.0]
Q_1     = [191.93]

sigma_2 = [0,      0.0132, -0.00363]
Q_2     = [20.167, 21.97,  80.23]

## F-SAC implementation

Now we implement the F-SAC model by deriving the basic `SAC` class and implementing the method calculating the interaction energies, `calc_u`, and its temperature derivative `calc_du_dT`.

Here we use **temperature dependent** interaction energies by the following multiplying factor:
\begin{equation}
f(T) = \exp \left (-\beta \frac{T-T_0}{1000 \cdot \text{K}} \right )
\end{equation}

This function returns 1 when $T=T_0$, being an exponential decay with the temperature.

In [44]:
# F-SAC parameters
alpha = 8544.6
Q_eff = 3.597

# The mixture area and sigma arrays
Q = [Q_1, Q_2]
sigma = [sigma_1, sigma_2]

T0 = 308.15
beta = 1

class FSACNonHB(SAC):
    def __init__(self):
        super().__init__(Q_eff=Q_eff)

    def calc_u(self, T, i, j, m, n):
        '''
        Interaction energies, only the electrostatic contribution (no hydrogen bonding)
        and temperature dependent.
        '''
        f = math.exp(-beta * (T - T0)/1000)
        return f*(alpha/2)*(sigma[i][m]+sigma[j][n])**2
    
    def calc_du_dT(self, T, i, j, m, n):
        '''
        Returns the derivative of the interaction energy.
        '''
        f = math.exp(-beta * (T - T0)/1000)
        u = self.calc_u(T, i, j, m, n)
        
        return -beta/1000 * u * f


# Create an instance of our F-SAC class
sac = FSACNonHB()

# Set a temperature and the areas of our compounds
T = T0
sac.set_compounds(Q)
sac.set_temperature(T)

### Check the excess energy

We can get the total internal and Helmholtz energies and subtract the pure values to get excess quantities. Additionally we can use standard thermodynamic relations to get the entropy.

We first do this comparison with $\beta = 0$ (interaction energies are not temperature dependent).

In [45]:
x = [0.4, 0.6]
beta = 0
sac.set_composition(x)
sac.calc_ln_gamma()

ue = sac.get_energy()
ae = sac.get_helmholtz()
for i in range(len(Q)):
    ue -= x[i] * sac.get_energy_pure(i)
    ae -= x[i] * sac.get_helmholtz_pure(i)

se_R = ue - ae

We can also calculate the entropy by deriving the Helmholtz energy and then calculate back the energy. The values should match, within the accuracy of our finite differences derivative.

In [46]:
dT = 0.1;
T2 = T + dT
sac.set_temperature(T2)
sac.calc_ln_gamma()

ae_T2 = sac.get_helmholtz()
for i in range(len(Q)):
    ae_T2 -= x[i] * sac.get_helmholtz_pure(i)

se_R_check = -T*(ae_T2 - ae)/dT - ae;
ue_check = ae + se_R_check

print(f"ae/RT: {ae}")
print(f"ue/RT: {ue}")
print(f"se/R: {se_R}")
print(f"ue/RT (check): {ue_check}")
print(f"se/R (check): {se_R_check}")

print(f"difference on ue/RT: {ue_check-ue}")
print(f"difference on se/R: {se_R_check-se_R}")


ae/RT: 0.4110998824992649
ue/RT: 0.5121568520653366
se/R: 0.10105696956607169
ue/RT (check): 0.511953072515641
se/R (check): 0.10085319001637616
difference on ue/RT: -0.00020377954969552636
difference on se/R: -0.00020377954969552636


Now we can do the same with $\beta = 1$.
The Hemlholtz excess energy should not change, since we are evaluating at the reference temperature $T_0$.
The excess internal energy and entropy should change.

In [47]:
beta = 1
sac.set_temperature(T)
sac.calc_ln_gamma()

ue = sac.get_energy()
ae = sac.get_helmholtz()
for i in range(len(Q)):
    ue -= x[i] * sac.get_energy_pure(i)
    ae -= x[i] * sac.get_helmholtz_pure(i)

se_R = ue - ae

The values calculated by finite differences should match again our new values.

In [48]:
dT = 0.1;
T2 = T + dT
sac.set_temperature(T2)
sac.calc_ln_gamma()

ae_T2 = sac.get_helmholtz()
for i in range(len(Q)):
    ae_T2 -= x[i] * sac.get_helmholtz_pure(i)

se_R_check = -T*(ae_T2 - ae)/dT - ae;
ue_check = ae + se_R_check

print(f"ae/RT: {ae}")
print(f"ue/RT: {ue}")
print(f"se/R: {se_R}")
print(f"ue/RT (check): {ue_check}")
print(f"se/R (check): {se_R_check}")

print(f"difference on ue/RT: {ue_check-ue}")
print(f"difference on se/R: {se_R_check-se_R}")

ae/RT: 0.4110999207325188
ue/RT: 0.6699780710468444
se/R: 0.2588781503143256
ue/RT (check): 0.6698595087236736
se/R (check): 0.25875958799115484
difference on ue/RT: -0.00011856232317075666
difference on se/R: -0.00011856232317075666
